In [12]:
# with open('RDE_2 JDLE-15.dat', 'r', encoding='ISO-8859-1') as archivo:
#     contenido = archivo.read()
#     print(contenido)


from asammdf import MDF
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import plotly.graph_objects as go
import plotly.express as px

### Nombres de documentos a procesar con datos OBD y GPS de INCA

#### Grabación Intervalo Autopista 15042024, con tiempo de grabación 17:57 minutos // Mayor vel prom
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Autopista_15042024/RDE_2 JDLE-15.dat"

#### Grabación Intervalo Rural 15042024, con tiempo de grabación 25:45 minutos // Mayor vel prom
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Rural_15042024/RDE_2 JDLE-14.dat"

#### Grabación Intervalos Rural + Autopista 05042024, con tiempo de grabación 46:08 minutos
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Rural_Autopista_05042024/RDE_1 JDLE-12.dat"

#### Grabación Intervalos Urbano 02042024, con tiempo de grabación 32:29 minutos
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_02042024/RDE_1 JDLE-07.dat"

#### Grabación Intervalos Urbano 15042024, con tiempo de grabación 30:26 minutos // Mayor vel prom
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_15042024/RDE_2 JDLE-13.dat"

#### Grabación Intervalos Urbano 19042024, con tiempo de grabación 42:53 minutos // Low
"C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_19042024/RDE_3_JDLE-20.dat"


In [13]:
# Carga el archivo .DAT
mdf = MDF('C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_19042024/RDE_3_JDLE-20.dat')

#Extraer los datos de las variables de interes de VEHICULO
datosRPM = mdf.get('PID0C_engineRPM')[:] 
datosVEL = mdf.get('PID0D_vehicleSpeedSensor')[:] 
datosTCO = mdf.get('PID05_engineCoolantTemperature')[:] 
datosTime = mdf.get('PID1F_timeSinceEngineStart')[:]

#Extraer los datos de las variables de interes de GPS
datoslat = mdf.get('Latitude')[:] 
datoslong = mdf.get('Longitude')[:] 
datosAltitude = mdf.get('Altitude')[:] 
datosUTC_Time = mdf.get('UTC_Time')[:]
datosSpeed = mdf.get('Speed')[:] 

In [14]:
# Tiempo de grabacion usando PID1F_timeSinceEngineStart ---- OBD
df1 = pd.DataFrame() #crear dataframe
df1['time'] = datosTime
t_record = (df1['time'].max() - df1['time'].min()) # Tiempo de grabacion en s
print (t_record , "segundos")
print ("Confirmar el tiempo de grabacion en min")
print (t_record/60 , "minutos")

2574 segundos
Confirmar el tiempo de grabacion en min
42.9 minutos


In [15]:
# Tiempo de grabacion usando UTC_Time ---- GPS
df2 = pd.DataFrame() #crear dataframe
df2['UTC_Time'] = datosUTC_Time
# Duracion_GPS = (df2['UTC_Time'].max() - df2['UTC_Time'].min())/1.667
# print (Duracion_GPS , "segundos")
# print ("Confirmar el tiempo de grabacion en min")
# print (Duracion_GPS/60 , "minutos")

In [16]:
# Crear un diccionario con los datos de los arrays
data_motor = {
    'RPM': datosRPM[:min(len(datosRPM), len(datosVEL), len(datosTCO))],
    'Speed': datosVEL[:min(len(datosRPM), len(datosVEL), len(datosTCO))],
    'TCO': datosTCO[:min(len(datosRPM), len(datosVEL), len(datosTCO))]}

# Crear el DataFrame de datos OBD motor
dfmotor = pd.DataFrame(data_motor)

# Crear el DataFrame de datos GPS
df_gps = pd.DataFrame()
df_gps['Latitude'] = datoslat
df_gps['Longitude'] = datoslong
df_gps['Altitude'] = datosAltitude
df_gps['Speed_gps'] = datosSpeed

# Se crea la columna de tiempo para datos Motor (vehiculo) y GPS
num_filas_obd = len(df1)
dfmotor['Tiempo'] = pd.Series([round(i * (t_record / (num_filas_obd - 1)), 4) for i in range(num_filas_obd)])
num_filas_gps = len(df2)
df_gps['Tiempo'] = pd.Series([round(i * (t_record / (num_filas_gps - 1)), 4) for i in range(num_filas_gps)])

In [6]:
# guardar datos OBD motor en un archivo CSV :
dfmotor.to_excel('C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_19042024/OBD_Urban_CR_19042024.xlsx', index=False)
# guardar datos GPS en un archivo CSV :
df_gps.to_excel('C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/01.Data_Acquisitions/Urban_19042024/GPS_Urban_CR_19042024.xlsx', index=False)

In [17]:
### INTERPOLAR DATOS PARA UNA FRECUENCIA DE 1Hz
# Crear un nuevo DataFrame con valores de tiempo enteros, teniendo en cuenta la duracion de adquisicion de datos
df_int = pd.DataFrame({"t_s": np.arange(0, t_record, 1)})
# Interpolar los valores de las variables para los valores enteros de tiempo en un UNICO Dataframe (DATOS OBD MOTOR)
df_int["v_km_h"] = np.interp(df_int["t_s"], dfmotor['Tiempo'], dfmotor['Speed']).round(2)
df_int["RPM"] = np.interp(df_int["t_s"], dfmotor['Tiempo'], dfmotor['RPM']).round(2)
df_int["TCO_C"] = np.interp(df_int["t_s"], dfmotor['Tiempo'], dfmotor['TCO']).round(2)

# Interpolar los valores de las variables para los valores enteros de tiempo en un UNICO Dataframe (DATOS GPS)
df_int["Latitude"] = np.interp(df_int['t_s'], df_gps['Tiempo'], df_gps['Latitude'])
df_int["Longitude"] = np.interp(df_int['t_s'], df_gps['Tiempo'], df_gps['Longitude'])
df_int["Altitude_msnm"] = np.interp(df_int['t_s'], df_gps['Tiempo'], df_gps['Altitude'])
df_int["v_km_h_gps"] = np.interp(df_int['t_s'], df_gps['Tiempo'], df_gps['Speed_gps'])


# Filtrado de la velocidad
x = np.array(df_int['v_km_h'])
Velfil = savgol_filter(x,5,3)  
Velfil[Velfil <= 0.5]=0
df_int['v_km_h_f'] = Velfil
df_int['v_km_h_f'] = df_int['v_km_h_f'].round(2)
df_int["v_m_s_f"] = df_int['v_km_h_f']/3.6

# Filtrado de la RPM
x = np.array(df_int['RPM'])
RPMfil = savgol_filter(x,5,3) 
# RPMfil[Velfil <= 0]=0
df_int['RPM_f'] = RPMfil
df_int['RPM_f'] = df_int['RPM_f'].round(2)


In [11]:
# ##### OJO para el caso donde se han grabado dos intervalos juntos. 
# ## Ubicar los puntos donde se separan los intervalos

# # Velocidad
# fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int["v_km_h_gps"], name = "v_km_h_gps", mode="lines"),
#                       go.Scatter(x = df_int["t_s"], y = df_int["v_km_h"], name = "v_km_h", mode="lines"),
#                       go.Scatter(x = df_int["t_s"], y = df_int["v_km_h_f"], name = "v_km_h_f", mode="lines",visible = "legendonly")])

# fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
# fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
# fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
# fig.show()


In [8]:
# ### PARA EL RURAL

# df_int_Rural = df_int.iloc[0:1714].copy()
# df_int_Rural['intervalo'] = "Rural"

# # Distancia acumulada en el RDE
# # Calcula el cambio en el tiempo
# df_int_Rural['delta_tiempo'] = df_int_Rural['t_s'].diff().fillna(0)
# # Calcula la distancia recorrida en cada intervalo de tiempo
# df_int_Rural['distance'] = df_int_Rural["v_m_s_f"] * df_int_Rural['delta_tiempo']
# # Calcula la distancia acumulada en km
# df_int_Rural['distance'] = (df_int_Rural['distance'].cumsum()) # en km

# # Puedes guardar este DataFrame en un archivo CSV de la siguiente manera:
# df_int_Rural.to_excel('Interval_Rural_05042024.xlsx', index=False)

# ### PARA EL MOTORWAY

# df_int_Auto = df_int.iloc[1715:2768].copy()
# df_int_Auto["t_s"] = df_int_Auto["t_s"] - 1715
# df_int_Auto['intervalo'] = "Motorway"

# # Distancia acumulada en el RDE
# # Calcula el cambio en el tiempo
# df_int_Auto['delta_tiempo'] = df_int_Auto['t_s'].diff().fillna(0)
# # Calcula la distancia recorrida en cada intervalo de tiempo
# df_int_Auto['distance'] = df_int_Auto["v_m_s_f"] * df_int_Auto['delta_tiempo']
# # Calcula la distancia acumulada en km
# df_int_Auto['distance'] = (df_int_Auto['distance'].cumsum()) # en km

# # Puedes guardar este DataFrame en un archivo CSV de la siguiente manera:
# df_int_Auto.to_excel('Interval_Motorway_05042024.xlsx', index=False)

In [22]:
df_int['intervalo'] = "Urban"#"Rural""Motorway" 

# Distancia acumulada en el RDE
import plotly.express as px
# Calcula el cambio en el tiempo
df_int['delta_tiempo'] = df_int['t_s'].diff().fillna(0)
# Calcula la distancia recorrida en cada intervalo de tiempo
df_int['distance'] = df_int["v_m_s_f"] * df_int['delta_tiempo']
# Calcula la distancia acumulada en km
df_int['distance'] = (df_int['distance'].cumsum()) # en km



In [9]:
# Puedes guardar este DataFrame en un archivo CSV de la siguiente manera:
df_int.to_excel('Interval_Urban_19042024.xlsx', index=False)
df_int

,t_s,v_km_h,RPM,TCO_C,Latitude,Longitude,Altitude_msnm,v_km_h_gps,v_km_h_f,v_m_s_f,RPM_f,intervalo,delta_tiempo,distance
0,0,0.0,770.50,61.0,38.996918,-3.919499,633.80000,0.012964,0.00,0.000000,770.58,Urban,0.0,0.000000
1,1,0.0,769.00,61.0,38.996918,-3.919499,633.80000,0.005556,0.00,0.000000,768.69,Urban,1.0,0.000000
2,2,0.0,768.00,61.0,38.996918,-3.919499,633.80000,0.007973,0.00,0.000000,768.47,Urban,1.0,0.000000
3,3,0.0,769.50,61.0,38.996918,-3.919499,633.80000,0.012886,0.00,0.000000,740.14,Urban,1.0,0.000000
4,4,0.0,770.00,61.0,38.996918,-3.919499,633.80000,0.014816,0.00,0.000000,852.29,Urban,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,2569,23.0,1452.46,96.0,38.990944,-3.902695,634.76317,23.316662,23.04,6.400000,1456.52,Urban,1.0,15418.680556
2570,2570,24.0,1513.54,96.0,38.990927,-3.902737,634.70000,23.929692,24.09,6.691667,1509.66,Urban,1.0,15425.372222
2571,2571,25.0,1550.23,96.0,38.990902,-3.902810,634.70000,24.740857,24.74,6.872222,1550.91,Urban,1.0,15432.244444
2572,2572,25.0,1584.23,96.0,38.990870,-3.902910,634.70000,25.536042,25.17,6.991667,1583.78,Urban,1.0,15439.236111


In [23]:
# RPM
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int["RPM"], name = "RPM", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["RPM_f"], name = "RPM_f", mode="lines",visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()


# Velocidad
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int["v_km_h_gps"], name = "v_km_h_gps", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["v_km_h"], name = "v_km_h", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["v_km_h_f"], name = "v_km_h_f", mode="lines",visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()

# Disnacia acumulada
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int['distance'], name = "distance", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["distance"], name = "distance", mode="lines", visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()

#GPS altimetria
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int['Altitude_msnm'], name = "Altitude_msnm", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["Altitude_msnm"], name = "Altitude_msnm", mode="lines", visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()

#GPS
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int['Latitude'], name = "Latitude", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["Longitude"], name = "Longitude", mode="lines", visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()

# TCO
fig = go.Figure(data=[go.Scatter(x = df_int["t_s"], y = df_int['TCO_C'], name = "TCO_C", mode="lines"),
                      go.Scatter(x = df_int["t_s"], y = df_int["TCO_C"], name = "TCO_C", mode="lines", visible = "legendonly")])

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Tiempo")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="")
fig.update_layout(height=300, width=900, title_text="_",plot_bgcolor="white")
fig.show()